In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
from tensorflow.keras import models

import matplotlib.pyplot as plt


In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
train_dir=r'C:\Users\kushal yadav\OneDrive\Desktop\deeplearning\train'
validation_dir=r'C:\Users\kushal yadav\OneDrive\Desktop\deeplearning\validation'
test_dir=r'C:\Users\kushal yadav\Desktop\OneDrive\deeplearning\test'

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
#generating batches of tensor image data
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
                                )
test_datagen=ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(150,150),
 
batch_size=20,
class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
validation_dir,
target_size=(150,150),
batch_size=20,
class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150,150,3))

In [12]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [13]:
from tensorflow.keras import layers
from tensorflow.keras import models


In [14]:
model= models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())


model.add(layers.Dense(256,activation='relu'))

model.add(layers.Dense(1,activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [17]:
conv_base.trainable=False

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [19]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc']
             )

In [20]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("vgg-{epoch:02d}.h5",save_best_only=True)

In [24]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
   epochs=30,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=[checkpoint_cb])

Epoch 1/30
100/100 [==============================] - 460s 5s/step - loss: 0.3540 - acc: 0.8440 - val_loss: 0.2687 - val_acc: 0.8910
Epoch 2/30
100/100 [==============================] - 425s 4s/step - loss: 0.3493 - acc: 0.8435 - val_loss: 0.2697 - val_acc: 0.8880
Epoch 3/30
100/100 [==============================] - 435s 4s/step - loss: 0.3432 - acc: 0.8510 - val_loss: 0.2586 - val_acc: 0.8900
Epoch 4/30
 41/100 [===========>..................] - ETA: 9:54:25 - loss: 0.3331 - acc: 0.8439 

KeyboardInterrupt: 

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
hist_df=pd.DataFrame(history.history)

In [ ]:
hist_csv_file='history.csv'
with open(hist_csv_file,mode='w')as f:
    hist_df.to_csv(f)

In [ ]:
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(150,150),
batch_size=20,
class_mode='binary')


In [ ]:
model.evaluate(test_generator,steps=50)